In [21]:
import pandas as pd
import os
import csv
import glob
import numpy as np

In [10]:
glob.glob("*.gff3")

['Mus_musculus.GRCm38.102.exon.gff3',
 'Mus_musculus.GRCm38.102.gff3',
 'Mus_musculus.GRCm38.102.mRNA.gff3']

In [53]:
use_chrs = [str(x) for x in list(range(1,20))] + ['X']

In [188]:
##### Remove chromosome Y and random & simplify Exon ID
gff_file = 'Mus_musculus.GRCm38.102.exon.gff3'
out_gff_file = gff_file.replace(".gff3", ".simp.gff3")
gff_df = pd.read_csv(gff_file, sep='\t', dtype={0:'str'}, header=None)

# Remove the ones that are not in chr1-19 or x 
gff_df = gff_df[[True if x in use_chrs else False for x in gff_df[0].tolist()]]

# Simplify to Exon ID
gff_df[8] = [x.split("Name=")[1].split(";")[0] for x in gff_df[8]]
gff_df[8] = ["ID=%s"%x for x in gff_df[8]]
gff_df.to_csv(out_gff_file, index=False, header=None, sep="\t", quoting=csv.QUOTE_NONE)

In [187]:
##### Remove chromosome Y and random & simplify Exon ID
gff_file = 'Mus_musculus.GRCm38.102.mRNA.gff3'
out_gff_file = gff_file.replace(".gff3", ".simp.gff3")
gff_df = pd.read_csv(gff_file, sep='\t', dtype={0:'str'}, header=None)

# Remove the ones that are not in chr1-19 or x 
gff_df = gff_df[[True if x in use_chrs else False for x in gff_df[0].tolist()]]

# Simplify to Exon ID
gff_df[8] = [x.split("Name=")[1].split(";")[0].split("-")[0] for x in gff_df[8]]
gff_df[8] = ["ID=%s"%x for x in gff_df[8]]
gff_df.to_csv(out_gff_file, index=False, header=None, sep="\t", quoting=csv.QUOTE_NONE)

In [186]:
##### For each gene keep one mRNA
out_rmdup_file = gff_file.replace(".gff3", ".simp.rmdup.gff3")
id_unique = np.unique(gff_df[8])
with open(out_rmdup_file, "w") as fout:
    wfout = csv.writer(fout, delimiter="\t")
    for id_i in id_unique:
        gff_df_id = gff_df[gff_df[8] == id_i]
        start = min(gff_df_id[3])
        end = max(gff_df_id[4])
        new_list = gff_df_id.iloc[0,:].tolist()
        new_list[1] = 'merged'
        new_list[3] = start
        new_list[4] = end
        wfout.writerow(new_list)